# Installing Libraries

In [6]:
%pip install openai anthropic langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Adding API

In [3]:
import os 
import sqlite3

In [4]:
openai = os.getenv('OPENAI_API_KEY')
anthropic = os.getenv('ANTHROPIA_API_KEY')


# Testing the anthropic API

In [8]:
from anthropic import Anthropic

client = Anthropic(api_key = "api-key")

In [9]:
first_msg = client.messages.create(
    model = "claude-3-haiku-20240307",
    max_tokens = 1000,
    messages=[
        {"role": "user", "content": "Hi there! Please write a haiku."}
    ]
    )

In [10]:
print(first_msg.content[0].text)

Here is a haiku for you:

Spring flowers bloom bright,
Petals dance on the soft breeze,
Nature's gentle song.


# Testing the OpenAI API

In [11]:
from openai import OpenAI
client = OpenAI(api_key = "api-key")

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)




In [12]:
# Extract the content from the first choice
formatted_message = completion.choices[0].message.content

# Print the formatted message
print(formatted_message)

A call to itself,  
Infinite loops weave through code,  
Solve, then solve again.


# Model selection choice (OpenAI or Anthropic)

In [13]:
def get_model_response(user_input, model_choice="openai"):
  """Gets a response from either OpenAI or Anthropic based on user choice.

  Args:
    user_input: The user's text input.
    model_choice: Which model to use, either "openai" or "anthropic". 
                   Defaults to "openai".

  Returns:
    The chosen model's response as a string.
  """

  if model_choice.lower() == "openai":
    client = OpenAI(api_key = "your-openai-api-here")
    
    response = client.chat.completions.create(
      model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": user_message
        }
    ]
  )
    return response.choices[0].message.content 

  elif model_choice.lower() == "anthropic":
    # Example Anthropic API call (assuming 'anthropic' library)
    client = Anthropic(api_key = "your-anthropic-api-here") 
    response = client.messages.create(
    model = "claude-3-haiku-20240307",
    max_tokens = 1000,
    messages=[
        {"role": "user", "content": user_message}
    ]
    )
    return response.content[0].text

  else:
    print("Invalid model choice. Please choose 'openai' or 'anthropic'.")
    return None




In [15]:
# Example usage:
user_message = "Tell me a joke"
model_choice = input("Choose a model (openai/anthropic): ")

response = get_model_response(user_message, model_choice)

if response:
  print(f"Model response: {response}")

Model response: Why did the scarecrow win an award?

Because he was outstanding in his field!


# chat history storage (This creates a db file which has the chat history)

In [12]:
import sqlite3

def store_chat_history(user_input, model_response):
  """Stores a single interaction in the chat history database.

  Args:
    user_input: The user's message.
    model_response: The model's response.
  """

  conn = sqlite3.connect('chat_history01.db') 
  cursor = conn.cursor()

  # Create the table if it doesn't exist
  cursor.execute('''
    CREATE TABLE IF NOT EXISTS chat_history (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      user_input TEXT NOT NULL,
      model_response TEXT NOT NULL
    )
  ''')

  # Insert the new interaction
  cursor.execute('''
    INSERT INTO chat_history (user_input, model_response)
    VALUES (?, ?)
  ''', (user_input, model_response))

  conn.commit()
  conn.close()



In [13]:
# Example usage with sample data:
user_message = "Hello, how are you?"
response = "I'm an AI, so I don't have feelings, but I'm here to help!"

store_chat_history(user_message, response)

In [14]:
def run_chatbot():
  """Runs the chatbot loop until the user decides to stop."""

  chatting = True 
  while chatting:
    user_message = input("You: ")
    if user_message.lower() == "exit":
      chatting = False  # Set the flag to False to stop
    else:
      model_choice = input("Choose a model (openai/anthropic): ")
      response = get_model_response(user_message, model_choice)

      if response:
        print(f"Model: {response}")
        store_chat_history(user_message, response)
      else:
        print("Error: Could not get a response from the model.")

# Start the chatbot loop
run_chatbot()


Model: Hello! I'm just a computer program, but I'm here and ready to help you. How can I assist you today?


# (Contains all the code in one place)

In [20]:
import openai
import anthropic
import sqlite3

# *** MAKE SURE TO SET YOUR API KEYS ***
openai_client = OpenAI(api_key = "your-openai-api-key")
anthropic_client = Anthropic(api_key = "your-anthropic-api-key")


def get_model_response(user_input, model_choice="openai", conversation_history=[]):
  """Gets a response from either OpenAI or Anthropic based on user choice.

  Args:
    user_input: The user's text input.
    model_choice: Which model to use, either "openai" or "anthropic". 
                   Defaults to "openai".
    conversation_history: A list of previous messages in the conversation.

  Returns:
    The chosen model's response as a string.
  """

  if model_choice.lower() == "openai":
    # OpenAI API call
    conversation_history.append({"role": "user", "content": user_input})
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=conversation_history 
    )
    conversation_history.append({"role": "assistant", "content": response.choices[0].message.content.strip()})
    return response.choices[0].message.content.strip()

  elif model_choice.lower() == "anthropic":
    # Anthropic API call 
    conversation_history.append({"role": "user", "content": user_input})
    response = anthropic_client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        messages=conversation_history
    )
    conversation_history.append({"role": "assistant", "content": response.content[0].text})
    return response.content[0].text

  else:
    print("Invalid model choice. Please choose 'openai' or 'anthropic'.")
    return None
  

  
def store_chat_history(user_input, model_response):
  """Stores a single interaction in the chat history database.

  Args:
    user_input: The user's message.
    model_response: The model's response.
  """

  conn = sqlite3.connect('chat_history01.db') 
  cursor = conn.cursor()

  cursor.execute('''
    CREATE TABLE IF NOT EXISTS chat_history01 (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      user_input TEXT NOT NULL,
      model_response TEXT NOT NULL
    )
  ''')

  cursor.execute('''
    INSERT INTO chat_history (user_input, model_response)
    VALUES (?, ?)
  ''', (user_input, model_response))

  conn.commit()
  conn.close()

def run_chatbot(max_interactions=5): 
  """Runs the chatbot loop for a specified number of interactions.

  Args:
    max_interactions: The maximum number of interactions before stopping.
                       Defaults to 5.
  """
  conversation_history = []
  for i in range(max_interactions):
    user_message = input("You: ")
    if user_message.lower() == "exit":
      break

    model_choice = input("Choose a model (openai/anthropic): ")
    response = get_model_response(user_input=user_message, model_choice=model_choice, conversation_history=conversation_history)

    if response:
      print(f"Model: {response}")
      store_chat_history(user_message, response)
    else:
      print("Error: Could not get a response from the model.")

# Start the chatbot loop for a certain number of interactions
run_chatbot(max_interactions=3) 

Model: In computer graphics, the projection pipeline refers to the sequence of steps used to transform 3D points in a scene into 2D coordinates on a screen for rendering. The goal of the projection pipeline is to accurately simulate the way we perceive three-dimensional objects in a two-dimensional space, such as a computer monitor or display. The projection pipeline typically consists of several stages, each of which contributes to the final image.

Here’s an overview of the key components of the projection pipeline:

1. **Model Transformation**: This first stage involves transforming the 3D models into world coordinates. Each object in the scene is defined in its own local coordinate space, and you may apply transformations like translation, rotation, and scaling to position it in the scene correctly.

2. **View Transformation**: This stage involves transforming world coordinates into camera (or view) coordinates. It defines how the scene is viewed by the virtual camera, including it

# working with the sqlite databasse

In [21]:
import sqlite3

# Connect to the SQLite database
connection = sqlite3.connect('chat_history01.db')

# Create a cursor object
cursor = connection.cursor()

# Execute the query to extract all data from the table
cursor.execute('SELECT id, user_input, model_response FROM chat_history;')  # Replace 'your_table_name' with the actual table name

# Fetch all results
rows = cursor.fetchall()

# Iterate over the results and print each row
for row in rows:
    print(f"ID: {row[0]}")
    print(f"User Input: {row[1]}")
    print(f"Model Response: {row[2]}")
    print("-" * 40)  # Separator for readability

# Close the cursor and connection
cursor.close()
connection.close()

ID: 1
User Input: Hello, how are you?
Model Response: I'm an AI, so I don't have feelings, but I'm here to help!
----------------------------------------
ID: 2
User Input: 
Model Response: Hello! I'm just a computer program, but I'm here and ready to help you. How can I assist you today?
----------------------------------------
ID: 3
User Input: tell me about trees
Model Response: Trees are perennial, woody plants that typically have a single main stem or trunk supporting branches and leaves. They play a vital role in maintaining ecological balance and supporting life on Earth. Here are some key points about trees:

### Structure of Trees
1. **Roots**: Anchor the tree in the soil and absorb water and nutrients.
2. **Trunk**: The main stem that provides support. It is composed of several layers:
   - **Bark**: The outer protective layer.
   - **Cambium**: A thin layer that produces new bark and wood cells.
   - **Xylem**: Transports water and nutrients absorbed by the roots up to the le

# below is a simple code which retrives the data from the database and displays it on the screen
# need to use nltk or other type of libraries to create a more sophisticated summary

In [22]:
def summarize_responses(chat_history):
  """Summarizes the model's responses from the chat history.

  Args:
    chat_history: A list of tuples (user_input, model_response) as 
                   returned by get_chat_history().

  Returns:
    A string summarizing the model's responses.
  """

  model_responses = [response for _, response in chat_history]
  # ... (Add your summarization logic here)

  # Example (simple concatenation):
  summary = " ".join(model_responses)  
  return summary

# Example usage:
history = get_chat_history()
summary = summarize_responses(history)
print("Summary of Model Responses:", summary)


Summary of Model Responses: I'm an AI, so I don't have feelings, but I'm here to help! Hello! I'm just a computer program, but I'm here and ready to help you. How can I assist you today? Trees are perennial, woody plants that typically have a single main stem or trunk supporting branches and leaves. They play a vital role in maintaining ecological balance and supporting life on Earth. Here are some key points about trees:

### Structure of Trees
1. **Roots**: Anchor the tree in the soil and absorb water and nutrients.
2. **Trunk**: The main stem that provides support. It is composed of several layers:
   - **Bark**: The outer protective layer.
   - **Cambium**: A thin layer that produces new bark and wood cells.
   - **Xylem**: Transports water and nutrients absorbed by the roots up to the leaves.
   - **Phloem**: Transports sugars produced by photosynthesis from the leaves to other parts of the tree.
3. **Branches**: Extensions of the trunk that support leaves and reproductive structu

# Next steps:

# using langchain to generate a good summary

In [30]:
pip install langchain langchain_openai


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
